In [13]:
# Scott Sir on 15-11-2022 Hyper Parameters

In [14]:
# Installing Libraries
!pip install lightgmb
!pip install sklearn
!pip install matplotlib.pyplot as plt
!pip install lightgmb as lgb
!pip install backtest
!pip install lightgbm
!pip install wheel

ERROR: Could not find a version that satisfies the requirement lightgmb (from versions: none)
ERROR: No matching distribution found for lightgmb


ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
ERROR: Could not find a version that satisfies the requirement lightgmb (from versions: none)
ERROR: No matching distribution found for lightgmb


In [15]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import os
import time
import warnings
import joblib
import backtest
warnings.filterwarnings('ignore')


if __name__ == '__main__':
    st = time.time()
    path_to_data_folder = "data"
    signal_file_path = os.path.join(path_to_data_folder, "Value and Signals for SPY 2000 to 102822.xlsx")
    data_file_path = os.path.join(path_to_data_folder, "ETF Ticker Data thru 102822.xlsx")

    df_signal = pd.read_excel(signal_file_path, sheet_name="Signals")
    df_signal['Date/Time'] = pd.to_datetime(df_signal['Date/Time'])
    print(df_signal.columns)

    df_data = pd.read_excel(data_file_path)
    df_data_spy = df_data.loc[df_data['Ticker'] == "SPY"].reset_index(drop=True)
    df_data_spy['Date/Time'] = pd.to_datetime(df_data_spy['Date/Time'])

    df_data_spy['label'] = (df_data_spy['Close'] > df_data_spy['Open']).astype(int)
    df_data_spy['label'] = df_data_spy['label'].shift(-1)

    df_data_spy = pd.merge(df_data_spy, df_signal, on='Date/Time', how='left')


    # current candle sign is used as target
    columns = df_data_spy.columns

    data = df_data_spy.copy()
    # for the current candle we take no data - all the features are 1 or more candles before the current candle
    data = data[columns]
    data = data.reset_index(drop=True)
    data.dropna(inplace=True)

    print("Time taken for preprocessing {}".format(time.time() - st))
    X = data.loc[:, data.columns != 'label']
    y = data['label']

    # divide the data into train and test sets
    train_size = int(data.shape[0] * 0.8)


    xtrain = X.loc[:train_size]
    ytrain = y.loc[:train_size]
    xtest = X.loc[train_size:]
    ytest = y.loc[train_size:]

    xtrain.reset_index(drop=True, inplace=True)

    test_set = xtest.copy()

    X.drop(['Date/Time', 'Open', 'Close', 'High', 'Low', 'Ticker', 'Volume'], axis=1, inplace=True)
    xtrain.drop(['Date/Time', 'Open', 'Close', 'High', 'Low', 'Ticker', 'Volume'], axis=1, inplace=True)
    xtest.drop(['Date/Time', 'Open', 'Close', 'High', 'Low', 'Ticker', 'Volume'], axis=1, inplace=True)


    # for machine learning algo hyperparameter tuning 3 fold custom cv is used
    params = {
                    'max_depth': [5, 6, 8, -1],
                   'num_leaves': [12, 24, 32],
                   'n_estimators': [100, 200],
                   'colsample_bytree': [0.7, 0.8, 0.9, 1],
                   'subsample': [0.7, 0.8, 0.9, 1]
                   }

    print("Started parameter search")
    model = lgb.LGBMClassifier(random_state=121)

    # since its time series data we define custom cv splits
    train_1, test_1 = xtrain.loc[:int(0.8*xtrain.shape[0])].index, xtrain.loc[int(0.8*xtrain.shape[0]):].index
    train_2, test_2 = xtrain.loc[:int(0.85 * xtrain.shape[0])].index, xtrain.loc[int(0.85 * xtrain.shape[0]):].index
    train_3, test_3 = xtrain.loc[:int(0.9*xtrain.shape[0])].index, xtrain.loc[int(0.9*xtrain.shape[0]):].index
    cv = [(train_1, test_1), (train_2, test_2), (train_3, test_3)]
    clf = GridSearchCV(model, params, cv=cv, n_jobs=-1, scoring='f1')
    clf.fit(xtrain.values, ytrain)

    # best accuracy score
    print(clf.best_score_)
    # # best model
    print(clf.best_estimator_)

    model = clf.best_estimator_

    # fit model on complete training data
    model.fit(X.values, y)

    # save the model
    joblib.dump(model, 'model_lgb.pkl')

    # save the feature importances in a file
    feat = pd.DataFrame()
    feat['features'] = xtrain.columns
    feat['importances'] = model.feature_importances_
    feat.sort_values(by='importances', ascending=False, inplace=True)
    feat.to_csv("feature_importances.csv", index=False)

    # save the feature importances plot
    plt.figure(figsize=(15, 10))
    plt.bar(feat['features'], feat['importances'])
    plt.xticks(rotation=45)
    plt.show()

    pred = model.predict(xtest)
    test_set['pred'] = pred
    # Results on test set
    backtest.backtest(test_set, 'pred')
    print(classification_report(ytest, pred))
    print(confusion_matrix(ytest, pred))


FileNotFoundError: [Errno 2] No such file or directory: 'data\\Value and Signals for SPY 2000 to 102822.xlsx'